In [1]:
import os
import tensorflow as tf
os.environ['KERAS_BACKEND'] = 'tensorflow'
#os.environ['CUDA_HOME'] = 'C:\\Users\\HT\\cuda'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

gpus = tf.config.list_physical_devices("GPU")
 
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [2]:
import numpy as np
import pandas as pd
#import tensorflow as tf
#from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Conv2D
#from keras.layers import Conv2D
from keras.layers import merge, Input
#from keras.engine import merge, Input, Model
#from keras.models import Model
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
#K.set_image_data_format('channels_last')

In [3]:
# nb_classes = 10

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# K.set_image_data_format('channels_last')
# # reorder dimensions for tensorflow
# #x_train = np.transpose(x_train.astype('float32') / 255., (0, 2, 3, 1))
# #x_test = np.transpose(x_test.astype('float32') / 255., (0, 2, 3, 1))
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
#import cv2
#import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import load_model
import os
import matplotlib.pyplot as plt
import seaborn as sns

#-----------------------------------------------------------------------------------------------------------
norm_size = 100
#datapath = 'C:\\Users\\HT\\Desktop\\论文\\Back\\data\\train'
EPOCHS = 50
INIT_LR = 1e-3
#labelList = []
#dicClass = {'cat': 0, 'dog': 1}
nb_classes = 2
#batch_size =160
#-------------------------------------------------------------------------------------------------------------
# 1: 加载数据文件，查看数据信息
df = pd.read_csv('train.csv')
df.head() # broadband 即可：0-离开，1-留存
df_test = pd.read_csv('test.csv')

from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
# LabelEncoder
for col in tqdm_notebook(df.columns):
    if df[col].dtype == "object":
        encoder = LabelEncoder()
        encoder.fit(list(df[col].values) + list(df_test[col].values))
        df[col] = encoder.transform(list(df[col].values))
        df_test[col] = encoder.transform(list(df_test[col].values))
        
df = df.reset_index() # 重置索引
df_test = df_test.reset_index()
def clean_top_cols(df):
    new_cols = [col for col in df.columns if df[col].value_counts(dropna=False, normalize=True).values[0] > 1]
    return new_cols
df_cols = clean_top_cols(df)
df_test_clos = clean_top_cols(df_test)
cols_to_drop = list(set(df_cols + df_test_clos))
if 'bad_good' in cols_to_drop : # 查看标签列是否在其中
    cols_to_drop.remove('bad_good') # 删除标签列
#print("原始的train shape : ", df)
df = df.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)
#print("清理后的train shape : ", df.shape)

y = df['bad_good'] # 标签
X1 = df.iloc[:, 1:-1] # 客户 id 没有用，故丢弃 cust_id



C:\Users\HT\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\HT\AppData\Local\Temp/ipykernel_5060/2065985837.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm_notebook(df.columns):


  0%|          | 0/627 [00:00<?, ?it/s]

In [4]:
print(X1.shape)
X1 = X1.drop('guozhai_flag',axis=1)
print(X1.shape)

(285285, 626)
(285285, 625)


In [6]:
y.shape

(285285,)

In [5]:
X1 = X1[0:30000]
y = y[0:30000]

In [6]:
X = np.array(X1, dtype="int")
#print(X1.shape)
X.resize(30000,25,25,3)
X = np.array(X, dtype='float')

In [9]:
print(X.shape)
#x_train =X

(30000, 25, 25, 3)


In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#Keras: 2.24
# Paper: Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning

from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization


def create(classes_num=2, image_height=28, image_width=28, image_channel=1):
    inputs = Input((image_height, image_width, image_channel))
    net = block_stem(inputs)
    for i in range(4):
        net = block_inception_a(net)
    net = block_reduction_a(net)
    for i in range(7):
        net = block_inception_b(net)
    net = block_reduction_b(net)
    for i in range(3):
        net = block_inception_c(net)
    net = AveragePooling2D((1, 1))(net)
    net = Dropout(0.2)(net)
    net = Flatten()(net)
    outputs = Dense(units=1, activation='sigmoid')(net)
    return Model(inputs, outputs, name='Inception-v4')
def block_stem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(1, 1), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))

    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='valid')(net)
    branch_2 = conv2d(net, 96, (3, 3), strides=(1, 1), padding='valid')
    net = concatenate([branch_1, branch_2])
    branch_1 = conv2d(net, 64, (1, 1))
    branch_1 = conv2d(branch_1, 96, (3, 3), padding='valid')
    branch_2 = conv2d(net, 64, (1, 1))
    branch_2 = conv2d(branch_2, 64, (7, 1))
    branch_2 = conv2d(branch_2, 64, (1, 7))
    branch_2 = conv2d(branch_2, 96, (3, 3), padding='valid')
    net = concatenate([branch_1, branch_2])
    branch_1 = conv2d(net, 192, (3, 3), strides=(1, 1), padding='valid')  # different from the paper
    branch_2 = MaxPooling2D((3, 3), strides=(1, 1), padding='valid')(net)
    net = concatenate([branch_1, branch_2])
    return net

def block_inception_a(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 96, (1, 1))
    branch_2 = conv2d(inputs, 96, (1, 1))
    branch_3 = conv2d(inputs, 64, (1, 1))
    branch_3 = conv2d(branch_3, 96, (3, 3))
    branch_4 = conv2d(inputs, 64, (1, 1))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    return concatenate([branch_1, branch_2, branch_3, branch_4])

def block_inception_b(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 128, (1, 1))
    branch_2 = conv2d(inputs, 384, (1, 1))
    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (1, 7))
    branch_3 = conv2d(branch_3, 256, (7, 1))  # different from the paper
    branch_4 = conv2d(inputs, 192, (1, 1))
    branch_4 = conv2d(branch_4, 192, (1, 7))
    branch_4 = conv2d(branch_4, 224, (7, 1))
    branch_4 = conv2d(branch_4, 224, (1, 7))
    branch_4 = conv2d(branch_4, 256, (7, 1))
    return concatenate([branch_1, branch_2, branch_3, branch_4])

def block_inception_c(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 256, (1, 1))
    branch_2 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(inputs, 384, (1, 1))
    branch_3_1 = conv2d(branch_3, 256, (1, 3))
    branch_3_2 = conv2d(branch_3, 256, (3, 1))
    branch_4 = conv2d(inputs, 384, (1, 1))
    branch_4 = conv2d(branch_4, 448, (1, 3))
    branch_4 = conv2d(branch_4, 512, (3, 1))
    branch_4_1 = conv2d(branch_4, 256, (3, 1))
    branch_4_2 = conv2d(branch_4, 256, (1, 3))
    return concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2])


def block_reduction_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='valid')(inputs)
    branch_2 = conv2d(inputs, 384, (3, 3), strides=(1, 1), padding='valid')
    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (3, 3))
    branch_3 = conv2d(branch_3, 256, (3, 3), strides=(1, 1), padding='valid')
    return concatenate([branch_1, branch_2, branch_3])


def block_reduction_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='valid')(inputs)
    branch_2 = conv2d(inputs, 192, (1, 1))
    branch_2 = conv2d(branch_2, 192, (3, 3), strides=(1, 1), padding='valid')
    branch_3 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(branch_3, 256, (1, 7))
    branch_3 = conv2d(branch_3, 320, (7, 1))
    branch_3 = conv2d(branch_3, 320, (3, 3), strides=(1, 1), padding='valid')
    return concatenate([branch_1, branch_2, branch_3])


def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    return net

if __name__ == '__main__':
    model = create()
    model.summary()

Model: "Inception-v4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 26, 26, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 26, 26, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 26, 26, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [11]:
#%%time
#K.set_image_data_format('channels_last')
#img_rows, img_cols = 25, 25
#img_channels = 3

# in original inception-v4, these are 4, 7, 3, respectively
#num_A_blocks = 1
#num_B_blocks = 1
#num_C_blocks = 1

#inputs = Input(shape=(img_rows, img_cols, img_channels))

# #x = inception_v4_stem(inputs)
# x = block_stem(inputs)
# for i in range(num_A_blocks):
#     #x = inception_v4_A(x)
#     x = block_inception_a(x)
# #x = inception_v4_reduction_A(x)
# x = block_reduction_a(x)
# for i in range(num_B_blocks):
#     #x = inception_v4_B(x)
#     x = block_reduction_b(x)
# #x = inception_v4_reduction_B(x)
# x = block_inception_b(x)
# for i in range(num_C_blocks):
#     #x = inception_v4_C(x)
#     x = block_inception_c(x)
# #model = tf.keras.Sequential()
# x = AveragePooling2D(pool_size=(1, 1), strides=(1, 1), padding='valid')(x)
# x = Dropout(0.5)(x)
# x = Flatten()(x)

#predictions = Dense(nb_classes, activation='softmax')(x)
#predictions = Dense(1, activation='softmax')(x)

#model = Model(inputs=inputs, outputs=predictions)

In [12]:
#model.summary()

In [8]:

%%time

model.compile(optimizer=Adam(learning_rate=INIT_LR),
              #loss='categorical_crossentropy',
              loss='binary_crossentropy', # Loss
              metrics=['accuracy'])

Wall time: 30 ms


In [9]:
x_train,x_test,y_train,y_test = train_test_split(X, y,test_size=0.3,shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_train.shape)

(21000, 25, 25, 3)
(9000, 25, 25, 3)
(9000,)
(21000,)


In [10]:
from matplotlib import pyplot as plt
batch_size = 150 #128
nb_epoch = 30
data_augmentation = False
from tensorflow.keras import datasets, Sequential, layers,metrics

# Model saving callback
#checkpointer = ModelCheckpoint(filepath='stochastic_depth_cifar10.hdf5', verbose=1, save_best_only=True)
#model = Sequential()
if not data_augmentation:
    print('Not using data augmentation.')
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1,
                        validation_data=(x_test, y_test),
                        shuffle=True,
                        callbacks=[])
else:
    print('Using real-time data augmentation.')

    # realtime data augmentation
    # datagen_train = ImageDataGenerator(
    #     featurewise_center=False,
    #     samplewise_center=False,
    #     featurewise_std_normalization=False,
    #     samplewise_std_normalization=False,
    #     zca_whitening=False,
    #     rotation_range=0,
    #     width_shift_range=0.125,
    #     height_shift_range=0.125,
    #     horizontal_flip=True,
    #     vertical_flip=False)
    # datagen_train.fit(x_train)

    # fit the model on the batches generated by datagen.flow()
    history = model.fit(x_train, y_train, batch_size=batch_size, shuffle=True,
                                  #steps_per_epoch=x_train.shape[0], 
                                  epochs=nb_epoch, verbose=1,
                                  validation_data=(x_test, y_test),
                                  callbacks=[])
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
   

Not using data augmentation.
Epoch 1/30
 45/140 [========>.....................] - ETA: 10:48:50 - loss: 0.7654 - accuracy: 0.9418

In [ ]:
    # plt.figure(figsize=(15, 5))
    # plt.subplot(1, 2, 1)
    # plt.plot(acc, label='Training Accuracy')
    # plt.plot(val_acc, label='Validation Accuracy')
    # plt.title('Training and Validation Accuracy')
    # plt.legend()
    # #plt.grid()
 
    # plt.subplot(1, 2, 2)
    # plt.plot(loss, label='Training Loss')
    # plt.plot(val_loss, label='Validation Loss')
    # plt.title('Training and Validation Loss')
    # plt.legend()
    #  #plt.grid()
    # plt.show()
 
    # #plot_acc_loss_curve(history)

Error: Kernel is dead